## Задание 1

Скачайте с сайта https://grouplens.org/datasets/movielens/ датасет любого размера. Определите какому фильму было выставлено больше всего оценок 5.0.

## Решение:

In [433]:
import pandas as pd

In [434]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
movies = pd.read_csv('ml-latest-small/movies.csv')

In [435]:
res = ratings[ratings['rating'] == 5.0]['movieId'].value_counts().index[0]
print(f'Фильм с максимальным количеством оценок 5.0 ({res}): {movies["title"][movies["movieId"] == res].iloc[0]}')

Фильм с максимальным количеством оценок 5.0 (318): Shawshank Redemption, The (1994)


## Задание 2

По данным файла power.csv посчитайте суммарное потребление стран Прибалтики (Латвия, Литва и Эстония) категорий 4, 12 и 21 за период с 2005 по 2010 года. Не учитывайте в расчетах отрицательные значения quantity.

## Решение:

In [436]:
data = pd.read_csv('power.csv')

Хотел сделать функцией (как в презентации), но не смог найти, как через apply передать в функцию несколько аргументов. Поэтому сделал длинным запросом.

In [437]:
filtered_data = data[ ((data['country'] == 'Estonia') | (data['country'] == 'Latvia') | (data['country'] == 'Lithuania')) & ((data['category'] == 4) | (data['category'] == 12) | (data['category'] == 21)) & (data['year'] <= 2010) & (data['year'] >= 2005) & (data['quantity'] > 0)]
sum(filtered_data['quantity'])

240580.0

## Задание 3

Выберите страницу любого сайта с табличными данными. Импортируйте таблицы в pandas dataframe.
Примеры страниц (необязательно брать именно эти):
https://fortrader.org/quotes
https://www.finanz.ru/valyuty/v-realnom-vremeni

## Решение:

In [438]:
page_url = 'https://www.finanz.ru/valyuty/v-realnom-vremeni'

На странице две таблицы, считываем их по отдельности

In [439]:
raw_quotes = pd.read_html(page_url, attrs = {'class': 'header_user_pricebox'}, decimal=',', thousands='', encoding='utf-8')[0]
currencies = pd.read_html(page_url, attrs = {'class': 'quote_list'}, decimal=',', thousands='', encoding='utf-8')[0].drop(columns='Unnamed: 0')

Таблицы выглядят страшновато, поэтому немного их причесал

In [440]:
import numpy as np

In [441]:
quotes = pd.DataFrame(raw_quotes.values.reshape(12,3))
quotes.columns = ['Индекс', 'Значение', 'Изменение']
quotes

,Индекс,Значение,Изменение
0,ММВБ,2 952,"-0,7%"
1,Nasdaq,10 793,"-1,1%"
2,Биткойн,10 950,"-1,3%"
3,USD/RUB,75.6995,"0,1%"
4,РТС,1 229,"-1,2%"
5,S&P500,3 319,"-1,1%"
6,Нефть,43.0,"-0,5%"
7,EUR/RUB,89.677,"0,8%"
8,Dow,27 657,"-0,9%"
9,FTSE100,6 007,"-0,7%"


Честно сказать, не смог ни найти, ни придумать, как заставить read_html воспринимать каждый th внути table как отдельную таблицу, поэтому сделал функцию для вытаскивания из DataFrame конкретной валюты

In [442]:
currencies.rename(columns={'Рубль':'Индекс'}, inplace=True)

In [443]:
def get_currency_rates(currency, currencies):
    if currency.upper() in ('RUB', 'CHF', 'UAH'):
        return currencies[currencies['Индекс'].str.contains(f'/{currency.upper()}')].drop_duplicates()
    else:
        return currencies[currencies['Индекс'].str.contains(f'{currency.upper()}/')].drop_duplicates()        

In [444]:
get_currency_rates('jpy', currencies)

,Индекс,Предложение,Спрос,Предыдущее закрытие,%,Абсолютное значение,Время
3,JPY/RUB,0.7240,-,0.7235,"0,06 %",0.0004,19.09.2020
17,JPY/EUR (100:1),0.8100,-,0.8100,"-0,03 %",0.0000,19.09.2020
18,JPY/GBP,0.0074,-,0.0074,"0,00 %",0.0000,19.09.2020
19,JPY/USD,0.0096,-,0.0096,"0,00 %",0.0000,19.09.2020
20,JPY/CHF (100:1),0.8700,-,0.8700,"0,54 %",0.0000,19.09.2020
